In [1]:
%load_ext autoreload
%aimport -tf
%aimport data
%aimport model
%autoreload 1

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import data
import model
import nltk

In [3]:
tf.logging.set_verbosity(tf.logging.INFO)

In [53]:
# Hyperparameters
MAX_DOCUMENT_LENGTH = 20
VOCAB_MIN_FREQUENCY = 50
DATA_TEST_SIZE = 5000

# Model Parameters
RNN_CELL_SIZE = 128
EMBEDDING_DIM = 128

# Training Parameters
TRAIN_BATCH_SIZE = 8
TRAIN_EVAL_EVERY = 1000

In [54]:
ds = data.load_reddit_data(
    max_document_length=MAX_DOCUMENT_LENGTH,
    min_frequency=VOCAB_MIN_FREQUENCY,
    test_size=DATA_TEST_SIZE)

In [55]:
data.print_dataset_stats(ds)

Vocabulary Size: 10803
Train Data Shape: (385367, 20)
Dev Data Shape (5000, 20)
Vocabulary size 10803


In [56]:
train_input_fn, dev_input_fn = data.create_train_dev_input_fns(ds, batch_size=TRAIN_BATCH_SIZE)

In [57]:
def simple_rnn(sequences, sequence_lengths):
    cell = tf.nn.rnn_cell.GRUCell(RNN_CELL_SIZE)
    sequence_list = tf.unpack(sequences, axis=1)
    return tf.nn.rnn(cell, sequence_list, dtype=tf.float32, sequence_length=sequence_lengths)

In [65]:
# Monitor for sampling sentences from the estimator
sample_mon = model.SentenceSampleMonitor(
    vocab=ds.vocab,
    every_n_steps=TRAIN_EVAL_EVERY,
    first_n_steps=-1)

# Monitor for development set loss
dev_monitor = tf.contrib.learn.monitors.ValidationMonitor(
    input_fn=dev_input_fn,
    max_sample_length=10,
    num_samples=5,
    every_n_steps=TRAIN_EVAL_EVERY)

# The function to create the model
model_fn = model.create_language_model_rnn(
    vocab_size=len(ds.vocab.vocabulary_),
    embedding_dim=EMBEDDING_DIM,
    rnn_fn=simple_rnn)

model_name = "maxlen_{}_rnn{}_embed_{}".format(MAX_DOCUMENT_LENGTH, RNN_CELL_SIZE, EMBEDDING_DIM)
estimator = tf.contrib.learn.Estimator(model_fn=model_fn, model_dir="./checkpoints/{}".format(model_name))

In [ ]:
estimator.fit(input_fn=train_input_fn, steps=None, monitors=[sample_mon, dev_monitor])

INFO:tensorflow:Create CheckpointSaver
INFO:tensorflow:Restored model from ./checkpoints/maxlen_20_rnn128_embed_128/model.ckpt-1800-?????-of-00001
